--> importing the necessary libraries

In [32]:
!pip install emoji

In [0]:
#emoji.UNICODE_EMOJI

In [0]:
import emoji
import pandas as pd
import numpy as np

In [36]:
emoji.emojize(":baseball:")

'⚾'

--> processing train and test data set

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train = pd.read_csv("train_emoji.csv" , header=None) 
test = pd.read_csv("test_emoji.csv" , header=None) 

In [39]:
train.head(10)

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN
5,I love you mum,0,NaN,NaN
6,Stop saying bullshit,3,NaN,NaN
7,congratulations on your acceptance,2,NaN,NaN
8,The assignment is too long,3,NaN,NaN
9,I want to go play,1,NaN,[3]


--> preparing the emojis to be used

In [0]:
emoji_dict = {
    0:":beating_heart:",
    1:":baseball:",
    2:":beaming_face_with_smiling_eyes:",
    3:":disappointed_face:",
    4:":fork_and_knife:"    
}

In [41]:
for ix in emoji_dict.values():
    print(emoji.emojize(ix))

💓
⚾
😁
😞
🍴


In [0]:
x_train = train[0]
y_train = train[1]

x_test = test[0]
y_test = test[1]

In [43]:
for i in range(5):
  print(x_train[i] , emoji.emojize(emoji_dict[y_train[i]]))

never talk to me again 😞
I am proud of your achievements 😁
It is the worst day in my life 😞
Miss you so much 💓
food is life 🍴


--> using glove vector method to obtain word embeddings 

In [0]:
embeddings = {}
with open('/content/drive/My Drive/glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')

        embeddings[word] = coeffs

In [45]:
embeddings["the"]

array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
       -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
        2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
        1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
       -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
       -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
        4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
        7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
       -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
        1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01],
      dtype=float32)

--> creating a function to give every word its embedding in a group of sentences 

In [0]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output 

In [47]:
x_trainemb = getOutputEmbeddings(x_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [48]:
x_test_emb = getOutputEmbeddings(x_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [49]:
print(x_trainemb.shape)
print(x_test_emb.shape)

(132, 10, 50)
(56, 10, 50)


--> creating the RNN architecture

In [0]:
from keras.layers import *
from keras.models import Sequential
from keras.utils import to_categorical

In [0]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [52]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.3))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

--> compiling the created RNN model

In [53]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])
model.fit(x_trainemb,y_train,batch_size=32,epochs=40,shuffle=True,validation_split=0.1)

Train on 118 samples, validate on 14 samples
Epoch 1/40
118/118 [==============================] - 2s 17ms/step - loss: 1.5823 - acc: 0.3390 - val_loss: 1.5800 - val_acc: 0.3571
Epoch 2/40
118/118 [==============================] - 0s 747us/step - loss: 1.5296 - acc: 0.3644 - val_loss: 1.5805 - val_acc: 0.3571
Epoch 3/40
118/118 [==============================] - 0s 760us/step - loss: 1.4854 - acc: 0.3729 - val_loss: 1.5860 - val_acc: 0.3571
Epoch 4/40
118/118 [==============================] - 0s 768us/step - loss: 1.4538 - acc: 0.3814 - val_loss: 1.5759 - val_acc: 0.2857
Epoch 5/40
118/118 [==============================] - 0s 768us/step - loss: 1.4251 - acc: 0.3983 - val_loss: 1.5137 - val_acc: 0.2857
Epoch 6/40
118/118 [==============================] - 0s 778us/step - loss: 1.3390 - acc: 0.5000 - val_loss: 1.4353 - val_acc: 0.3571
Epoch 7/40
118/118 [==============================] - 0s 765us/step - loss: 1.2658 - acc: 0.5339 - val_loss: 1.3446 - val_acc: 0.5000
Epoch 8/40
118/118

--> testing the model on the test set

In [55]:
model.evaluate(x_test_emb,y_test)

56/56 [==============================] - 0s 410us/step


[1.3857608011790685, 0.589285705770765]

In [0]:
pred = model.predict_classes(x_test_emb)

-->showing some predicted results

In [62]:
for i in range(30):
    print(' '.join(x_test[i]))
    print("expected : " + emoji.emojize(emoji_dict[np.argmax(y_test[i])]))
    print("predicted : " + emoji.emojize(emoji_dict[pred[i]]) + "\n")    

I want to eat
expected : 🍴
predicted : 🍴

he did not answer
expected : 😞
predicted : 😞

he got a raise
expected : 😁
predicted : 😞

she got me a present
expected : 💓
predicted : 💓

ha ha ha it was so funny
expected : 😁
predicted : 😁

he is a good friend
expected : 💓
predicted : 😁

I am upset
expected : 💓
predicted : 😞

We had such a lovely dinner tonight
expected : 💓
predicted : 😁

where is the food
expected : 🍴
predicted : 🍴

Stop making this joke ha ha ha
expected : 😁
predicted : 😁

where is the ball
expected : ⚾
predicted : ⚾

work is hard
expected : 😞
predicted : 😁

This girl is messing with me
expected : 😞
predicted : 💓

are you serious ha ha
expected : 😁
predicted : 😞

Let us go play baseball
expected : ⚾
predicted : ⚾

This stupid grader is not working
expected : 😞
predicted : 😞

work is horrible
expected : 😞
predicted : 😞

Congratulation for having a baby
expected : 😁
predicted : 😁

stop messing around
expected : 😞
predicted : 😞

any suggestions for dinner
expected : 🍴
predicted